In [39]:
import pyphen
import string
import re
import collections

In [2]:
pyphen.language_fallback("hu_HU")

'hu_HU'

In [3]:
'hu_HU' in pyphen.LANGUAGES

True

In [3]:
hypher = pyphen.Pyphen(lang='hu_HU')

In [4]:
hypher.inserted('almáspite')

'al-más-pi-te'

In [6]:
hypher.inserted('asszonnyal')

'asz-szony-nyal'

In [7]:
hypher.wrap("megszentségteleníthetetlenségeskedéseitekért", 30)

('megszentségteleníthetetlensé-', 'geskedéseitekért')

In [8]:
for pair in hypher.iterate("megszentségteleníthetetlenségeskedéseitekért"):
    print(pair)

('megszentségteleníthetetlenségeskedéseite', 'kért')
('megszentségteleníthetetlenségeskedései', 'tekért')
('megszentségteleníthetetlenségeskedése', 'itekért')
('megszentségteleníthetetlenségeskedé', 'seitekért')
('megszentségteleníthetetlenségeske', 'déseitekért')
('megszentségteleníthetetlenséges', 'kedéseitekért')
('megszentségteleníthetetlensé', 'geskedéseitekért')
('megszentségteleníthetetlen', 'ségeskedéseitekért')
('megszentségteleníthetet', 'lenségeskedéseitekért')
('megszentségteleníthe', 'tetlenségeskedéseitekért')
('megszentségtelenít', 'hetetlenségeskedéseitekért')
('megszentségtele', 'níthetetlenségeskedéseitekért')
('megszentségte', 'leníthetetlenségeskedéseitekért')
('megszentség', 'teleníthetetlenségeskedéseitekért')
('megszent', 'ségteleníthetetlenségeskedéseitekért')
('meg', 'szentségteleníthetetlenségeskedéseitekért')


In [9]:
hypher.inserted("megszentségteleníthetetlenségeskedéseitekért")

'meg-szent-ség-te-le-nít-he-tet-len-sé-ges-ke-dé-se-i-te-kért'

In [11]:
hypher.positions("almáspite")

[2, 5, 7]

In [56]:
def hyph_class(word, hypher=pyphen.Pyphen(lang='hu_HU'), aslist=False):
    """Hyphenating classification of the characters in the word. {B(egin),M(iddle),E(nd),S(ingle)}"""
    if(len(word)==0):
        raise IndexError("0 length word")
    ret = list('M' * len(word))
    ret[0]='B'
    ret[-1]='E'
    for i in hypher.positions(word):
        ret[i]='B'
        if(ret[i-1]=='B'):
            ret[i-1]='S'
        else:
            ret[i-1]='E'
    if(aslist):
        return ret
    return "".join(ret)
def same_char_num(word, hypher=pyphen.Pyphen(lang='hu_HU')):
    """Return true if the hyphenated word has as many chars as the original"""
    return len(hypher.inserted(word))==len(word)+len(hypher.positions(word))
def cleaning(data):
    """Text cleaning:
        lower the letters
        punctuation, digits ellimination"""
    formated_data = data.lower()
    formated_data = re.sub('['+string.punctuation+']','',formated_data)
    formated_data = re.sub('['+string.digits+']','',formated_data)
    return formated_data

In [7]:
hyph_class('asszonnyal',hypher)

'BEBMMEBMME'

In [8]:
hypher.inserted("asszonnyal")

'asz-szony-nyal'

In [9]:
hyph_class("megszentségteleníthetetlenségeskedéseitekért")

'BMEBMMMEBMEBEBEBMEBEBMEBMEBEBMEBEBEBESBEBMME'

In [6]:
hypher.inserted("autóval")

'au-tó-val'

In [57]:
counter_hu_data = collections.Counter()
word_list = []
c_all = 0
c_same_char_num = 0
with open('web2.2-freq-sorted.txt','r',errors='ignore',encoding='latin2') as f:
    i = 0
    for line in f:
        i = i+1
        words = line.split()
        if len(words)>1:
            if(words[1].isdigit()):
                counter_hu_data[cleaning(words[0])]+=int(words[1])
        if i>100000:
            break
for words in counter_hu_data.most_common():
    c_all+=1
    next_word = words[0]
    if(len(next_word)!=0 and same_char_num(next_word)):
        c_same_char_num+=1
        word_list.append([next_word,hyph_class(next_word)])

In [58]:
c_same_char_num/c_all

0.9867324640395745

In [60]:
print(word_list[100:200])

[['ember', 'BEBME'], ['ezek', 'BMME'], ['tovább', 'BEBMME'], ['vagyok', 'BEBMME'], ['ma', 'BE'], ['miatt', 'BEBME'], ['this', 'BMME'], ['mellett', 'BMEBMME'], ['vannak', 'BMEBME'], ['by', 'BE'], ['alapján', 'BMMEBME'], ['lenne', 'BMEBE'], ['with', 'BMME'], ['that', 'BMME'], ['tehát', 'BEBME'], ['te', 'BE'], ['it', 'BE'], ['ben', 'BME'], ['egész', 'BMMME'], ['néhány', 'BEBMME'], ['milyen', 'BEBMME'], ['át', 'BE'], ['nekem', 'BEBME'], ['előtt', 'BMMME'], ['ezzel', 'BEBME'], ['mivel', 'BEBME'], ['ezen', 'BMME'], ['nélkül', 'BMEBME'], ['lett', 'BMME'], ['stb', 'BME'], ['you', 'BME'], ['viszont', 'BEBMMME'], ['év', 'BE'], ['teljes', 'BMEBME'], ['erre', 'BEBE'], ['hiszen', 'BEBMME'], ['másik', 'BEBME'], ['ebben', 'BEBME'], ['mind', 'BMME'], ['der', 'BME'], ['három', 'BEBME'], ['talán', 'BEBME'], ['valami', 'BEBEBE'], ['die', 'BME'], ['bár', 'BME'], ['are', 'BME'], ['legyen', 'BEBMME'], ['tudom', 'BEBME'], ['from', 'BMME'], ['inkább', 'BEBMME'], ['során', 'BEBME'], ['például', 'BMEBEBE'], ['r

In [18]:
len(word_list)

2